## Task 1a.  Converting PDF voter data to CSV
### Reading PDF
### Text processing
### Data Frames

In [ ]:
'''Task 1. Write a script to collect all websites mentioned in the two PDFs, 
Part15 PX2 Declaration WIllis GoDaddy.pdf and Domains (Name.com).pdf
'''
#pip install git+https://github.com/deanmalmgren/textract.git -Install on notebook
#Install on terminal the following
#/usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"
#brew install tesseract
#brew install imagemagick


In [1]:
from IPython.core.display import HTML
style = """
<style>
div.output_area {
	overflow-y: scroll;
	max-height: 500px;
}
</style>
"""
HTML(style)

In [357]:
import requests
import urllib
import bs4
from bs4 import BeautifulSoup
import itertools
import gzip
import urllib.request as urllib2
import pandas as pd
import numpy as np
import PyPDF2
import textract
import re
import xlsxwriter


In [358]:
'''The pyDPF2 did not work today'''

filename ='Guntur-Mangalagiri-3.pdf' 

pdfFileObj = open(filename,'rb')               #open allows you to read the file
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)   #The pdfReader variable is a readable object that will be parsed
num_pages = pdfReader.numPages                 #discerning the number of pages will allow us to parse through all the pages

print(num_pages)
count = 2
text = ""
all_pages = {}
                                                            
while count < num_pages:                       #The while loop will read each page
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()
    all_pages[count] = text
    


39


In [359]:
'''So I went for textract. textract is ideal as it is efficent with pdf'''
text = textract.process(filename, method='tesseract', language='eng')

In [360]:
# printing text as byte object
text

b'ELECTORAL ROLL 2020 S01 Andhra Pradesh\n\n \n\nNo. Name and Reservation Status of Assembly Constituency : 87 - Mangalagiri\n\n(GEN)\n\n \n\nPart number\n3\n\n \n\nNo. Name and Reservation Status of Parliamentary Constituency(ies) in which the Assembly\nConstituency is located : 13 - Guntur\n\n1. Details of Revision\n\n \n\nYear of Revision\nQualifying Date\n\nType of revision\n\n2020 Roll Identification\nBasic Roll of Special Summary Revision-2019,\n01-01-2020 integrated with all its Supplements including\n\n. | Continuous Updation preceding Special Summary\nSpecial Summary Revision Revision-2020 and supplement-1 of additions,\n\n2020\n\nDate of Publication 14-02-2020\n\ndeletions and modifications under SSR-2020\n\n \n\n2. Details of part and polling area\n\n \n\nNo. and name of\n\nsections in the part\n\n \n\n1. NEAR KANAKA DURGAMMA GUDI\n\n2. NEAR ANJANEYA TEMPLE Main Town/Village : Penumaka\n3. NEAR VINAYAKA TEMPLE Police Station\n\n4. NEAR YSR BOMMA Mandal\n\n5. NEAR GANDHI BOMM

In [361]:

# Decode Text from byte as string

text = text.decode("utf-8")

In [28]:
# Printed neatly
print(text)

ELECTORAL ROLL 2020 S01 Andhra Pradesh

 

No. Name and Reservation Status of Assembly Constituency : 87 - Mangalagiri

(GEN)

 

Part number
3

 

No. Name and Reservation Status of Parliamentary Constituency(ies) in which the Assembly
Constituency is located : 13 - Guntur

1. Details of Revision

 

Year of Revision
Qualifying Date

Type of revision

2020 Roll Identification
Basic Roll of Special Summary Revision-2019,
01-01-2020 integrated with all its Supplements including

. | Continuous Updation preceding Special Summary
Special Summary Revision Revision-2020 and supplement-1 of additions,

2020

Date of Publication 14-02-2020

deletions and modifications under SSR-2020

 

2. Details of part and polling area

 

No. and name of

sections in the part

 

1. NEAR KANAKA DURGAMMA GUDI

2. NEAR ANJANEYA TEMPLE Main Town/Village : Penumaka
3. NEAR VINAYAKA TEMPLE Police Station

4. NEAR YSR BOMMA Mandal

5. NEAR GANDHI BOMMA CENTER Revenue Division

6. NEAR NAGENDRA SWAMY GUDI Distri

In [362]:
#Save the file as voter details as a text file
file2write=open("voter_details.txt",'w')
file2write.write(text)
file2write.close()

In [363]:
'''Building a dictionary of chunks of the 3 voter ids.Later on these chunks can 
be parsed again and formed the required details. The voter_modules dictionary 
will have the chunks of 3 voter ids. The first chunk will have all the details from the
start of the pdf. Other chunks will be neatly organized'''

flist = open("voter_details.txt").readlines()
parsing = False
block = ""
count = 0
voter_modules = {}
for line in flist:
    if line[0].isdigit():
        parsing = True
    
    if parsing:
        block+=line
        voter_modules[count]=block
    
    if line.endswith("Available\n"):
        #print(line)
        parsing = False
        block=""
        count+=1
    

In [364]:
len(voter_modules)

302

In [365]:
voter_modules[2]

"7 NYT1093004 p NYT1913490 3 CPT2315463,\nName: VASANTHI BELLAPU. Name: KAVITHA BELLAPU. Name: Subbarao Pasupuleti\nFather's Name : SRINIVASARAO- Father's Name : SRINIVASARAO- Father's Name : Fakiraiah\nBELLAPU BELLAPU House Number: 1-10\nHouse Number: 1-1/A Photo is House Number: 1-1/A Photo is Age: 47 Gender: MALE Photo is\nAge: 29 Gender: FEMALE Available Age: 22 Gender: FEMALE Available Available\n"

In [366]:
#Printing some sample chunks
print(voter_modules[39])

118 NYTO451237 119 NYT1593441 120 NYTO569798
Name : SATHYANARAYAN MUPPERA Name : SAMBASIVA RAO KUKKALA Name : VAJRAM KUKKALA
Father's Name Father's Name : VENKATESWARLU Husband's Name : SAMBASIVARAO_
VENKATESHWARARAO- KUKKALA House Number : 1-129
House Number: 1-128 Photo is House Number : 1-129 Photo is Age: 59 Gender: FEMALE Photo is
Age: 34 Gender: MALE Available Age: 63 Gender: MALE Available Available



In [367]:
#Printing some sample chunks
print(voter_modules[1])

4 NYT2140044 5 NYT2140028 5 NYT2140010
Name = KISHORBAABU CHEKKA Name : CHEKKALAKSHMI Name : CHEKKA CHANDRAVATHI
Father's Name: VENKATAPPAIAH Husband's Name : SADASHIV AR Husband's Name : KISHORUBAABU
House Number = 1-1 House Number = 1-1 House Number = 1-1
Age: 46 Gender: MALE Photo is Age: 46 Gender: FEMALE Photo is Age: 43 Gender: FEMALE Photo is
Available Available Available



In [368]:
print(voter_modules[202])

599 NYT0463125 600 NYT0623819 601 NYT2142586
Name: CHETAN KUMAR REDDY Name: PRAVALLIKA KALLAM Name: BukkaSamudram RaamSeetha
KALLAM Father's Name: KRISHNA REDDY Husband's Name : Ramireddy
Father's Name : KRISHNA REDDY House Number : 1-93 House Number: 1-4/4
House Number: 1-93, Photo is |] Age: 28 Gender: FEMALE Photo is |] Age: 49 Gender: FEMALE Photo is
Age: 31 Gender: MALE Available Available Available



In [402]:
# Cleaning a Chunks
def data_cleaning(dict_value):
    
    cleaned_dict = dict_value.replace('Photo is |]', '')
    cleaned_dict = cleaned_dict.replace("Father's Name", 'Mkr')
    cleaned_dict = cleaned_dict.replace("Husband's Name", 'Mkr')
    cleaned_dict = cleaned_dict.replace("House Number", 'Mkr')
    cleaned_dict = cleaned_dict.replace("Photo is", '')
    cleaned_dict = cleaned_dict.replace("Photois", '')
    cleaned_dict = cleaned_dict.replace("Available", '')
    cleaned_dict = cleaned_dict.replace("Name", 'Mkr')
    cleaned_dict = cleaned_dict.replace("Gender", 'Mkr')
    cleaned_dict = cleaned_dict.replace("Age", 'Mkr')
    cleaned_dict = cleaned_dict + "Mkr"
    cleaned_dict = (re.sub('(:*)+:',':',cleaned_dict))
    cleaned_dict = cleaned_dict.replace('\r', '').replace('\n', '')
    
    
    #cleaned_dict = re.findall(r"(?<=$)(.*?)(?=$)", cleaned_dict)
    print((cleaned_dict))
    

    return cleaned_dict


cleaned_data = {}

for key, value in voter_modules.items():
    cleaned_data[key] = data_cleaning(value)
    


3 No. Mkr and Reservation Status of Parliamentary Constituency(ies) in which the AssemblyConstituency is located : 13 - Guntur1. Details of Revision Year of RevisionQualifying DateType of revision2020 Roll IdentificationBasic Roll of Special Summary Revision-2019,01-01-2020 integrated with all its Supplements including. | Continuous Updation preceding Special SummarySpecial Summary Revision Revision-2020 and supplement-1 of additions,2020Date of Publication 14-02-2020deletions and modifications under SSR-2020 2. Details of part and polling area No. and name ofsections in the part 1. NEAR KANAKA DURGAMMA GUDI2. NEAR ANJANEYA TEMPLE Main Town/Village : Penumaka3. NEAR VINAYAKA TEMPLE Police Station4. NEAR YSR BOMMA Mandal5. NEAR GANDHI BOMMA CENTER Revenue Division6. NEAR NAGENDRA SWAMY GUDI District7. NEAR NTR BOMMA Pin Code  3. Polling Station Details            No. and Mkr of Polling Station : Type of Polling Station GENERAL(Male/Female/General)3-PENUMAKAAddress of Polling Station : N

In [403]:
print(cleaned_data[1])

4 NYT2140044 5 NYT2140028 5 NYT2140010Mkr = KISHORBAABU CHEKKA Mkr : CHEKKALAKSHMI Mkr : CHEKKA CHANDRAVATHIMkr: VENKATAPPAIAH Mkr : SADASHIV AR Mkr : KISHORUBAABUMkr = 1-1 Mkr = 1-1 Mkr = 1-1Mkr: 46 Mkr: MALE  Mkr: 46 Mkr: FEMALE  Mkr: 43 Mkr: FEMALE   Mkr


In [479]:
def convert_dict_to_list(value):
    

    

    
       
    identity_no = re.findall(r'\b[NYT|CPT]\w+', value)
    pattern = re.compile(r'(Mkr? )(\w) ')
    #[x.replace('Mkr', '') for x in identity_no]
    
    
    #print(identity_no)
    my_voter_list = re.findall(r"(?<=Mkr)(.*?)(?=Mkr)", value)
    my_voter_list = map(lambda each:each.strip(": "), my_voter_list)
    my_voter_list = list(my_voter_list)
    my_voter_list = [re.sub('[^a-zA-Z0-9-]+', '', _) for _ in my_voter_list]
    my_voter_list = [*identity_no,*my_voter_list]
    print(my_voter_list)
    #print(len(len(my_voter_list)))
    
    return my_voter_list

voter_list = {}

for key, value in cleaned_data.items():
    #print(value)
    voter_list[key] = convert_dict_to_list(value)


    

['No', 'Parliamentary', 'Constituency', 'Year', 'Continuous', 'Publication', 'No', 'NEAR', 'NEAR', 'TEMPLE', 'Town', 'Penumaka3', 'NEAR', 'TEMPLE', 'Police', 'NEAR', 'YSR', 'NEAR', 'CENTER', 'NEAR', 'NAGENDRA', 'NEAR', 'NTR', 'Pin', 'Code', 'Polling', 'No', 'Polling', 'Type', 'Polling', 'PENUMAKAAddress', 'Polling', 'Number', 'PollingStations', 'Parishad', 'Primary', 'Penumaka', 'PENUMAKA4', 'NUMBER', 'Net', 'No', 'No', 'Third', 'Total1', 'Pages', 'Page', 'Constituency', 'No', 'Part', 'No', 'PARISHAD', 'PRIMARY', 'PENUMAKA', 'TADEPALLI', 'CONSTITUENCY', 'Publication', 'Total', 'Pages', 'Page', 'Constituency', 'No', 'No', 'NEAR', 'NYT2140036', 'NYT2140051', 'NYT1067843Mkr', 'CHEKKAKANAKAMMA', 'CHEKKA', 'PARVATHI', 'CHOWDARY', 'PALEPUMkr', 'andReservationStatusofParliamentaryConstituencyiesinwhichtheAssemblyConstituencyislocated13-Guntur1DetailsofRevisionYearofRevisionQualifyingDateTypeofrevision2020RollIdentificationBasicRollofSpecialSummaryRevision-201901-01-2020integratedwithallitsSup

In [470]:
for key, value in voter_list.items():
    #print(value)
    print(value)

['No', 'Parliamentary', 'Constituency', 'Year', 'Continuous', 'Publication', 'No', 'NEAR', 'NEAR', 'TEMPLE', 'Town', 'Penumaka3', 'NEAR', 'TEMPLE', 'Police', 'NEAR', 'YSR', 'NEAR', 'CENTER', 'NEAR', 'NAGENDRA', 'NEAR', 'NTR', 'Pin', 'Code', 'Polling', 'No', 'Polling', 'Type', 'Polling', 'PENUMAKAAddress', 'Polling', 'Number', 'PollingStations', 'Parishad', 'Primary', 'Penumaka', 'PENUMAKA4', 'NUMBER', 'Net', 'No', 'No', 'Third', 'Total1', 'Pages', 'Page', 'Constituency', 'No', 'Part', 'No', 'PARISHAD', 'PRIMARY', 'PENUMAKA', 'TADEPALLI', 'CONSTITUENCY', 'Publication', 'Total', 'Pages', 'Page', 'Constituency', 'No', 'No', 'NEAR', 'NYT2140036', 'NYT2140051', 'NYT1067843Mkr', 'CHEKKAKANAKAMMA', 'CHEKKA', 'PARVATHI', 'CHOWDARY', 'PALEPUMkr', 'andReservationStatusofParliamentaryConstituencyiesinwhichtheAssemblyConstituencyislocated13-Guntur1DetailsofRevisionYearofRevisionQualifyingDateTypeofrevision2020RollIdentificationBasicRollofSpecialSummaryRevision-201901-01-2020integratedwithallitsSup

In [471]:
c=[voter_data for key,voter_data in voter_list.items()]
c[1:-1]

[['NYT2140044',
  'NYT2140028',
  'NYT2140010Mkr',
  'CHEKKA',
  'CHEKKALAKSHMI',
  'CHEKKA',
  'CHANDRAVATHIMkr',
  'KISHORBAABUCHEKKA',
  'CHEKKALAKSHMI',
  'CHEKKACHANDRAVATHI',
  'VENKATAPPAIAH',
  'SADASHIVAR',
  'KISHORUBAABU',
  '1-1',
  '1-1',
  '1-1',
  '46',
  'MALE',
  '46',
  'FEMALE',
  '43',
  'FEMALE'],
 ['NYT1093004',
  'NYT1913490',
  'CPT2315463',
  'PasupuletiMkr',
  'VASANTHIBELLAPU',
  'KAVITHABELLAPU',
  'SubbaraoPasupuleti',
  'SRINIVASARAO-',
  'SRINIVASARAO-',
  'FakiraiahBELLAPUBELLAPU',
  '1-10',
  '1-1A',
  '1-1A',
  '47',
  'MALE',
  '29',
  'FEMALE',
  '22',
  'FEMALE'],
 ['CPT2318731',
  'NYT0041608',
  'NYT0583815Mkr',
  'PADMA',
  'PASUPULETI',
  'NAGAMALLESWARA',
  'PICHIREDDYMkr',
  'PADMAPASUPULETI',
  'JANEEMIYSASHAIK',
  'NAGAMALLESWARAREDDY',
  'SUBBARAO',
  'HUSSENSAHEBAMUNNANGI',
  '1-10',
  '1-11',
  'PICHIREDDY',
  '42',
  'FEMALE',
  '48',
  'MALE',
  '1-11',
  '42',
  'MALE'],
 ['NYT0957365',
  'NYT0041376',
  'NYT0957373Mkr',
  'PRATAP',
  

In [472]:
''' 
Pass dictionary in Dataframe constructor to create a new object
keys will be the column names and lists in values will be column data
'''
dfObj = pd.DataFrame(c) 

In [473]:

dfObj = dfObj.drop(dfObj.index[0])
dfObj.head()

,0,1,2,3,4,5,6,7,8,9,...,83,84,85,86,87,88,89,90,91,92
1,NYT2140044,NYT2140028,NYT2140010Mkr,CHEKKA,CHEKKALAKSHMI,CHEKKA,CHANDRAVATHIMkr,KISHORBAABUCHEKKA,CHEKKALAKSHMI,CHEKKACHANDRAVATHI,...,None,None,None,None,None,None,None,None,None,None
2,NYT1093004,NYT1913490,CPT2315463,PasupuletiMkr,VASANTHIBELLAPU,KAVITHABELLAPU,SubbaraoPasupuleti,SRINIVASARAO-,SRINIVASARAO-,FakiraiahBELLAPUBELLAPU,...,None,None,None,None,None,None,None,None,None,None
3,CPT2318731,NYT0041608,NYT0583815Mkr,PADMA,PASUPULETI,NAGAMALLESWARA,PICHIREDDYMkr,PADMAPASUPULETI,JANEEMIYSASHAIK,NAGAMALLESWARAREDDY,...,None,None,None,None,None,None,None,None,None,None
4,NYT0957365,NYT0041376,NYT0957373Mkr,PRATAP,PRATAP,PITCHI,MUNNANGIBHANUPRATAP,KASIMBISHAIK,MUNNANGIJYOTHIREDDY,JANEEMAIAH,...,None,None,None,None,None,None,None,None,None,None
5,NYT0462820,NYT1807023,NYT2142644Mkr,PASUPULETI,Poornima,RAMBABUPASUPULETI,PoornimaMunnangi,MANIKALRAOMUPEERA,SUBBARAO,Somasekharareddy,...,None,None,None,None,None,None,None,None,None,None


In [474]:
dfObj = dfObj.drop(list(dfObj)[18:93], axis=1)
dfObj.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
1,NYT2140044,NYT2140028,NYT2140010Mkr,CHEKKA,CHEKKALAKSHMI,CHEKKA,CHANDRAVATHIMkr,KISHORBAABUCHEKKA,CHEKKALAKSHMI,CHEKKACHANDRAVATHI,VENKATAPPAIAH,SADASHIVAR,KISHORUBAABU,1-1,1-1,1-1,46,MALE
2,NYT1093004,NYT1913490,CPT2315463,PasupuletiMkr,VASANTHIBELLAPU,KAVITHABELLAPU,SubbaraoPasupuleti,SRINIVASARAO-,SRINIVASARAO-,FakiraiahBELLAPUBELLAPU,1-10,1-1A,1-1A,47,MALE,29,FEMALE,22
3,CPT2318731,NYT0041608,NYT0583815Mkr,PADMA,PASUPULETI,NAGAMALLESWARA,PICHIREDDYMkr,PADMAPASUPULETI,JANEEMIYSASHAIK,NAGAMALLESWARAREDDY,SUBBARAO,HUSSENSAHEBAMUNNANGI,1-10,1-11,PICHIREDDY,42,FEMALE,48
4,NYT0957365,NYT0041376,NYT0957373Mkr,PRATAP,PRATAP,PITCHI,MUNNANGIBHANUPRATAP,KASIMBISHAIK,MUNNANGIJYOTHIREDDY,JANEEMAIAH,BHANUPRATAP,PITCHIREDDY,1-11REDDY,1-11,40,FEMALE,1-11,40
5,NYT0462820,NYT1807023,NYT2142644Mkr,PASUPULETI,Poornima,RAMBABUPASUPULETI,PoornimaMunnangi,MANIKALRAOMUPEERA,SUBBARAO,Somasekharareddy,KRISHNAIAH,1-11Munningi,1-100,32,MALE,1-12,72,MALE


In [475]:
dfObj.columns = ['Voter_1','Voter_2','Voter_3','Name_1','Name_2','Name_3','Relative_1','Relative_2','Relative_3','House_1','House_2','House_3',
                     'Age_3','Gender_3','Age_2','Gender_2','Age_1','Gender_1']
dfObj.head(n=10)

,Voter_1,Voter_2,Voter_3,Name_1,Name_2,Name_3,Relative_1,Relative_2,Relative_3,House_1,House_2,House_3,Age_3,Gender_3,Age_2,Gender_2,Age_1,Gender_1
1,NYT2140044,NYT2140028,NYT2140010Mkr,CHEKKA,CHEKKALAKSHMI,CHEKKA,CHANDRAVATHIMkr,KISHORBAABUCHEKKA,CHEKKALAKSHMI,CHEKKACHANDRAVATHI,VENKATAPPAIAH,SADASHIVAR,KISHORUBAABU,1-1,1-1,1-1,46,MALE
2,NYT1093004,NYT1913490,CPT2315463,PasupuletiMkr,VASANTHIBELLAPU,KAVITHABELLAPU,SubbaraoPasupuleti,SRINIVASARAO-,SRINIVASARAO-,FakiraiahBELLAPUBELLAPU,1-10,1-1A,1-1A,47,MALE,29,FEMALE,22
3,CPT2318731,NYT0041608,NYT0583815Mkr,PADMA,PASUPULETI,NAGAMALLESWARA,PICHIREDDYMkr,PADMAPASUPULETI,JANEEMIYSASHAIK,NAGAMALLESWARAREDDY,SUBBARAO,HUSSENSAHEBAMUNNANGI,1-10,1-11,PICHIREDDY,42,FEMALE,48
4,NYT0957365,NYT0041376,NYT0957373Mkr,PRATAP,PRATAP,PITCHI,MUNNANGIBHANUPRATAP,KASIMBISHAIK,MUNNANGIJYOTHIREDDY,JANEEMAIAH,BHANUPRATAP,PITCHIREDDY,1-11REDDY,1-11,40,FEMALE,1-11,40
5,NYT0462820,NYT1807023,NYT2142644Mkr,PASUPULETI,Poornima,RAMBABUPASUPULETI,PoornimaMunnangi,MANIKALRAOMUPEERA,SUBBARAO,Somasekharareddy,KRISHNAIAH,1-11Munningi,1-100,32,MALE,1-12,72,MALE
6,NYT2142636,NYT2142669,NYT2142651Mkr,Chapidhi,Chappidi,MUPPERA,AanjaneyuluChapidhi,ChappidiSitharavammaVENKATSHIVLAKSHMI,Venkaiah,Aanjaneyulu,MANIKYALARAO-,1-101,1-101,1-100,69,MALE,57,FEMALE
7,CPT2315307,NYT0228718,NYT0230300Mkr,Chappidi,THIRUMALAR,CHAPPIDI,CHAPPIDIMkr,THIRUMALARMkr,VenkateshwararaoChappidi,THIRUMALARCHAPPIDI,SAMPOORENCHAPPIDI,Aanjaneyulu,-AANJANEYULU,THIRUMALAR,1-101,1-101,1-101,37
8,NYT2142677,NYT2142701,NYT2142693Mkr,Chappidi,ChappidiMangamheshwari,SubaraoMannava,MannavaSatyavathi,Sambashivarao,Kotaiah,Subbarao,1-102,1-103,1-103,59,FEMALE,62,MALE,56
9,NYT0230318,NYT2142727,NYT2142719Mkr,PRASAD,PedaVenkatareddi,ANOOSHARAVULAPALLI,KallamDhanamma,AllaMasemma,VENKATAPRASAD,PedaVenkatareddi,Seetaramireddy,1-103,1-104,1104,34,FEMALE,84,FEMALE
10,NYT2142743,NYT2142735,NYT1068064Mkr,PASAMMkr,PASAMMkr,SambireddyAlla,SubaAreddyAlla,SIVAKOTIREDDYPASAM,Seetaramireddy,Seetaramireddy,RAMIREDDYPASAM,1-104,1-104,1-104,44,MALE,43,MALE


In [481]:
dfObj.to_csv("Guntur-Mangalagiri-3.csv")

In [2]:
pip install minecraft

    ERROR: Complete output from command python setup.py egg_info:
    ERROR: Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/private/var/folders/mn/rn7yw42j6n13x1yqc3xm75400000gn/T/pip-install-xtk95rtl/minecraft/setup.py", line 9, in <module>
        with open(NOTICE, 'r', encoding='utf-8') as f:
    NotADirectoryError: [Errno 20] Not a directory: '/private/var/folders/mn/rn7yw42j6n13x1yqc3xm75400000gn/T/pip-install-xtk95rtl/minecraft/setup.py/../../NOTICE.rst'
    ----------------------------------------
ERROR: Command "python setup.py egg_info" failed with error code 1 in /private/var/folders/mn/rn7yw42j6n13x1yqc3xm75400000gn/T/pip-install-xtk95rtl/minecraft/
Note: you may need to restart the kernel to use updated packages.
